Please feel free to install if any other packages if required

In [1]:
!pip3 install -q langchain
!pip3 install -q transformers==4.35.0
!pip3 install -q accelerate
!pip3 install -q bitsandbytes

###### Required packages

In [2]:
import torch
import socket
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import PromptTemplate



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Check if GPU exists

In [3]:
torch.cuda.is_available()

True

##### Kaggle mounted model

In [4]:
model_name = '/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True,)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Loading model and tokenizer of text generation

In [7]:
pipe = pipeline("text-generation", 
                model=model,
                tokenizer = tokenizer, 
                torch_dtype=torch.bfloat16, 
                device_map="auto")

###### Sample Prompt

In [8]:
prompt = "Can you explain the concept of regularization in machine learning?"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you explain the concept of regularization in machine learning?

Comment: I'm not sure if this is the right place to ask this. I'll look around for more info and if I don't find any I'll ask on a different site. Thanks!

## Answer (0)

I found the answer here: https://stats.stackexchange.com/questions/128800/why-is-regularization-called-regularization

The answer is that


<a href="https://huggingface.co/docs/transformers/main_classes/text_generation">Text generation parameters to fine-tune</a>

LLM paramters to fine-tune
<ul>
    <li><a href="https://michaelehab.medium.com/the-secrets-of-large-language-models-parameters-how-they-affect-the-quality-diversity-and-32eb8643e631">link1</a></li>
    <li><a href="https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc">link2</a></li>
    <li><a href="https://replicate.com/replicate/oasst-sft-1-pythia-12b/versions/28d1875590308642710f46489b97632c0df55adb5078d43064e1dc0bf68117c3">Sample website to play with</a></li>
    <li></li>
</ul>


In [9]:
def get_output(prompt, 
               pipe = pipe,
               num_beams=1, 
               max_length=256,
               max_new_tokens=256, 
               do_sample=True,
               temperature=0.2,
               top_k=40,
               top_p=0.2, 
               repetition_penalty=1.1, 
               no_repeat_ngram_size=0, 
               stop_sequence=None,
               num_return_sequences=1,
               low_memory=True):
    try:
        stop_seq = None
        if stop_sequence is not None and type(stop_sequence)==list and len(stop_sequence)>0:
            text_encodings = []
            for word in stop_sequence:
                if type(word)==str:
                    words = word.strip().split()
                    if len(words)==1 and len(words[0].strip())>0:
                        text_encodings.append([words[0].strip()])
                    elif len(words)>1:
                        for w in words:
                            w = w.strip()
                            if len(w)>0:
                                text_encodings.append([w])
#             print(text_encodings)
            if len(text_encodings)>0:
                stop_seq = tokenizer(text_encodings,
                                     padding="max_length", is_split_into_words=True, truncation=True, return_tensors="pt") 
                            

#         print(stop_seq)
        output = pipe(prompt,num_beams = num_beams,
                      do_sample = do_sample,
                      max_length = max_length,
                      max_new_tokens = max_new_tokens, 
                      temperature = temperature, 
                      top_k = top_k, 
                      top_p = top_p,
                      repetition_penalty = repetition_penalty,
                      no_repeat_ngram_size = no_repeat_ngram_size,
                      # fix the stop_sequence to stoping_criteria so it would stop post that token.
#                       stop_sequence = stop_seq,
                      num_return_sequences=num_return_sequences,
                      low_memory=low_memory)
        return output
    except Exception as ex:
        print(f"Error while running llm: {ex}")
    return

    
    

Sample Ecommerce Prompt

In [10]:
%%time
prompt_ecom = "Generate a spellbinding Description using only the input Keywords. Don't use Price details, "\
"HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Description.\n"\
"Keywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, "\
"Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China\n Description:"
output = get_output(prompt_ecom)
print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Generate a spellbinding Description using only the input Keywords. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Description.\nKeywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China\n Description: The Fire-Boltt Phoenix Smart Watch is a stylish and functional timepiece that offers a range of features for users who want to stay connected and organized on the go. With its 1.3 inch display, this smart watch provides clear and easy-to-read information, making it perfect for checking the time, receiving notifications, and accessing other important data. The watch also comes equipped with 240 Watts Lithium Ion batteries, ensuring long battery life and reliable performance. Additionally, the Fire-Boltt Phoenix Smart Watch is compatible with Bluetooth connectivity, allowing us

Sample prompt job description generation

In [12]:
%%time
prompt_job = "Generate a spellbinding Job Description as a Hiring Manager using only the input Job Title,"\
" Experience and Skills. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, "\
"or information not in the input Keywords to generate the Job Description.\nKeywords: Machine Learning Engineer"\
" - NLP, 3+ years of experience, Natural Language Processing(NLP), LSTM, RNN, Neural nets, LLMs, ML algorithms,"\
" Data Structures, Algorithms, Python programming\n Description:"
job_output = get_output(prompt_job, 
                        max_length=512, 
                        max_new_tokens=512, 
                        num_beams=2, 
                        top_k=40,
                        temperature = 0.3,
                        top_p=0.3,
                        stop_sequence=['###',"##","```"],
                        no_repeat_ngram_size=0
                         )
for ind,out_exp in enumerate(job_output):
    print(f"Prompt:\n")

    print("Output generated:"
    print(out_exp["generated_text"],"\n")

Setting `pad_token_id` to `eos_token_id`:2787 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt:
Generate a spellbinding Job Description as a Hiring Manager using only the input Job Title, Experience and Skills. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Job Description.
Keywords: Machine Learning Engineer - NLP, 3+ years of experience, Natural Language Processing(NLP), LSTM, RNN, Neural nets, LLMs, ML algorithms, Data Structures, Algorithms, Python programming
 Description:

Output generated:
Machine Learning Engineer - NLP

We are looking for a Machine Learning Engineer with experience in Natural Language Processing (NLP) to join our team. You will be responsible for developing and implementing machine learning models to extract insights from text data. You will work closely with other engineers, data scientists, and product managers to build and deploy models that improve our products and services.

Responsibilities:

- Develop and implement machine learning models to extract insi

###### Please fine-tune the parameters as per your requirements and experiment